<a href="https://colab.research.google.com/github/linyu3294/cs6220-data-minning-hw/blob/main/cs6220_hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **HW7** **Social Graphs, Recommendation Systems**

Make sure you check the syllabus for the due date. Please use the notations adopted in class, even if the problem is stated in the book using a different notation.

We are not looking for very long answers (if you find yourself writing more than one or two pages of typed text per problem, you are probably on the wrong track). Try to be concise; also keep in mind that good ideas and explanations matter more than exact details.

Submit all code files Dropbox (create folder HW1 or similar name). Results can be pdf or txt files, including plots/tabels if any.
"Paper" exercises: submit using Dropbox as pdf, either typed or scanned handwritten.

---
DATATSET MovieLens 100K Ratings https://grouplens.org/datasets/movielens/100k/

DATATSET Netflix Prize ratings Dataset https://www.kaggle.com/netflix-inc/netflix-prize-data

DATATSET Friendster Social Graph http://socialcomputing.asu.edu/datasets/Friendster

DATATSET Flicker Social Graph http://networkrepository.com/soc-Flickr-ASU.php, but use the one curated in DM resources



---
## **PROBLEM 1: Recommender System using Collaborative Filtering**
Implement a Movie Recommendation System and run it on the Movie Lens Dataset (Train vs Test). Mesure performance on test set using RMSE

>First you are required to compute first a user-user similarity based on ratings and movies in common





In [62]:
import numpy as np



def get_user_data (path) :
  users = dict()
  with open(path, 'r') as file:
      lines = file.read().splitlines()

      for line in lines:
        user_id = int(line.split('\t') [0]) - 1
        movie_id = int(line.split('\t')[1]) - 1 
        rating = int(line.split('\t')[2])
      
        user_ratings = users.get(user_id, [])
        user_ratings.append((movie_id, rating))
        users[user_id] = user_ratings
  return users

def normalize_users_data (users):
  total = 0
  users_norm = dict()
  users_avg = []   
  users_variance = []
  for user_id in users.keys():
    ratings = [component[1] for component in users.get(user_id)]
    ratings_sum = np.sum(ratings)
    avg_rating = ratings_sum/len(ratings)
    users_avg.append(avg_rating)
    user_variance = (ratings_sum**2 / len(ratings) ) - avg_rating**2
    users_variance.append(user_variance)
    normalized_ratings = [(r[0],(r[1] - avg_rating) / user_variance) for r in users.get(user_id)]
    users_norm[user_id] = normalized_ratings
  return users_avg , users_variance, users_norm


def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


def get_users_similarities (users_norm) :
  users_similarity = []
  for u in range(len(users_norm)):
    temp_list = []
    for v in range(len(users_norm)):
      u_movies = [i[0] for i in users_norm[u]]
      v_movies = [i[0] for i in users_norm[v]]
      u_v_common_movies = intersection (u_movies, v_movies)
      u_common_rum = [i[1] for i in users_norm[u] if i[0] in u_v_common_movies]
      v_common_rum = [i[1] for i in users_norm[v] if i[0] in u_v_common_movies]
      if len(u_v_common_movies) > 0:
        similarity = np.dot(u_common_rum, v_common_rum) / len(u_v_common_movies)
      else :
        similarity = 0.0000001
      temp_list.append(similarity)
    users_similarity.append(temp_list)
  users_similarity = np.array(users_similarity)
  return (users_similarity)





In [63]:
movies = []
users = dict()
ratings = dict()
num_of_movies = 1682
num_of_users = 943

training_path = '/content/drive/MyDrive/NEU ALIGN CS Masters/CS6220 Data Mining/Colab Notebooks/ml-100k/u1.base'
test_path = '/content/drive/MyDrive/NEU ALIGN CS Masters/CS6220 Data Mining/Colab Notebooks/ml-100k/u1.test'


train_users= get_user_data(training_path)
test_users =get_user_data(test_path)
train_users_avg, train_users_variance, train_users_norm =  normalize_users_data(train_users)
train_users_similarity = get_users_similarities (train_users_norm)
print (train_users_similarity.shape)



(943, 943)


---
>Second, make rating predictions on the test set followoing the KNN idea: a prediction (user, movie) is the weighted average of other users' rating for the movie, weighted by user-similarity to the given user.

In [113]:
def find_other_users_with_movie_rating( users, users_avg, user, movie):
  other_users = []
  for key, ratings in users.items():
    # print (ratings)
    for component in ratings:
      if (component[0] == movie):
        other_users.append(key)
  return other_users
  
    

def predict (users_similarity, train_users_norm, users_avg, users_variance, user, movie):
  num = 0
  denom = 0

  other_users = find_other_users_with_movie_rating(train_users_norm, users_avg, user, movie)
  # print ('\npredicting user', user, '\'s rating for movie ', movie)
  # print ('Other users who also rated move ', movie, '\n' ,other_users , '\n\n')
  for other in other_users:
      for component in train_users_norm.get(other):
        if (component[0] == movie):
          other_rating = component[1]
          num += users_similarity[user][other] * other_rating
          denom += abs(users_similarity[user][other])

  # print (num)
  # print (denom)
  # print (users_avg[user])
  # print ((num)/(denom))
  # print (users_variance[user])
  if denom !=0:
    return users_avg[user] + ( (num)/(denom) ) * users_variance[user]
  else:
    print('No other user ratings.')





In [124]:

predicted_user = 29

print ('user train data', train_users.get(predicted_user), '\n', 'user test data', test_users.get(predicted_user))
for component in test_users.get(predicted_user):
    prediction = predict (train_users_similarity,
                          train_users_norm,
                          train_users_avg,
                          train_users_variance,
                          predicted_user,
                          component[0])
    prediction = round(prediction)
    print ( '\n', 'prediction of user ', predicted_user, 'for movie ', component[0],  ' : ', prediction )

    print ('actual user ', predicted_user, ' rating for movie ', component[0], ' : ',   component[1], '\n')

user train data [(1, 3), (28, 3), (49, 3), (68, 5), (134, 5), (160, 4), (163, 4), (171, 4), (173, 5), (241, 5), (251, 3), (256, 4), (257, 5), (258, 4), (303, 4), (312, 5), (314, 4), (318, 4), (320, 4), (402, 2), (682, 3), (750, 3), (779, 4), (872, 1), (891, 4)] 
 user test data [(6, 4), (27, 4), (81, 4), (180, 4), (230, 2), (254, 4), (285, 5), (288, 2), (293, 4), (300, 4), (434, 5), (530, 5), (537, 4), (538, 3), (677, 2), (687, 3), (1006, 5), (1012, 3)]

 prediction of user  29 for movie  6  :  4
actual user  29  rating for movie  6  :  4 


 prediction of user  29 for movie  27  :  4
actual user  29  rating for movie  27  :  4 


 prediction of user  29 for movie  81  :  4
actual user  29  rating for movie  81  :  4 


 prediction of user  29 for movie  180  :  4
actual user  29  rating for movie  180  :  4 


 prediction of user  29 for movie  230  :  4
actual user  29  rating for movie  230  :  2 


 prediction of user  29 for movie  254  :  4
actual user  29  rating for movie  254 

---
## **PROBLEM 2: EXTRA CREDIT Netflix Recommendations**


Implement a recommender system on the Netflix Prize Dataset. Use the "probe" set for testing. For a compettitive systems, one need to add movie content features such as actors, genres, directors, music, etc. These features are not available from Netflix, but for some movies they have been crawled by Movie Title form other websites such as IMDB (for example "movie_details.xml" file in DM_resources, but you can get more such features on your own.)



---
## **PROBLEM 3: Social Community Detection**

Implement a community detection algorithm on the Flicker Graph. Use the betweenes idea on edges and the Girvan–Newman Algorithm. The original dataset graph has more than 5M edges; in DM_resources there are 4 different sub-sampled graphs with edge counts from 2K to 600K; you can use these if the original is too big.

You should use a library to support graph operations (edges, vertices, paths, degrees, etc). We used igraph in python which also have builtin community detection algorithms (not allowed); these are useful as a way to evaluate communities you obtain

```
https://igraph.org/
```


---
## **PROBLEM 4: Knowledge Base Question Answering** ##

Given is knowledge graph with entities and relations, questions with starting entity and answers, and their word embedding . For each question, navigate the graph from the start entiry outwards until you find appropriate answer entities.
- there might be multiple answers correct, use F1 to evaluate
- utils functions (similarity, load_graphs) are given, but you can choose not to use them
- answers are given to be used for evaluation only
- your strategy should be a graph traversal augmented with scoring of paths; you might discard paths not promising along the way. This is similar to a focused crawl strategy.
- for simplicity, the questions are picked so that the answer is always at the end of the relevant path (not intermediary)

